In [1]:
#//*************************************************************************
#//*** Downloads Stock Data and converts the returns to a dataframe,
#//*** which saves a compressed CSV file in the stocks folder
#//*************************************************************************
import json
import requests
import time
import pandas as pd
import platform
import csv

#//*********************************************************************************
#//*** Read the API keys from a JSON encoded file
#//*** Located in the ignore_folder sub directory
#//*** This Folder is added to the .gitignore file and does not show up on Github
#//*** This is Authentication Best Practices for Github
#//*********************************************************************************
f = open("./ignore_folder/alpha_vantage_api.json", "r")

#//*** Fugley Pythonic type conversion
#//*** Loads the file into Dictionary via JSON.loads
#//*** Gets the API key value using the 'api' key
#//*** prepends apikey= so the resulting value is URL ready :]
av_apikey = json.loads(f.read())['apikey']
f.close()

#//*** Load the Stock Tickers
f = open(".\\data\\stock_tickers.json", "r")
symbols = json.loads(f.read())['symbols']

#//*** Just get amc for testing
symbols = [ "amc" ]
f.close()


In [4]:
"""
#//*** Alpha Vantage API Docs:
#//*** https://www.alphavantage.co/documentation/

#//*** Intra day Query
#symbol = "amc"
#url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={av_apikey}'

#//*** get prices throughout today
#action = "TIME_SERIES_INTRADAY"
#action = "TIME_SERIES_DAILY"
#//*** Intraday prices going back two yeares
#action = "TIME_SERIES_INTRADAY_EXTENDED"
#url = f'https://www.alphavantage.co/query?function={action}&symbol={symbol}&interval=60min&slice=year1month1&apikey={av_apikey}'
#url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=60min&slice=year1month3&adjusted=false&apikey={av_apikey}"
"""

#//******************************************************************************
#//*** Builds the URL request based on the symbol and type of data requested.
#//*** Initially, this does the daily numbers.
#//*** Can easily be scaled up to add many different URL request types
#//******************************************************************************
def build_url(input_action,input_symbol,m=1,y=1):
    #//*** Valid Actions:
    #//*******  Daily: Gets the historical daily closing price for up to 20 years
    
    if input_action == 'daily':
        action = "TIME_SERIES_DAILY"
        out = ""
        out += f'https://www.alphavantage.co/query?'
        out += f'function={action}'
        out += f'&symbol={symbol}'
        out += f'&outputsize=full'
        out += f'&apikey={av_apikey}'
        
        return out
    
    if input_action == '1min':
        action = "TIME_SERIES_INTRADAY_EXTENDED"
        out = ""
        out += f'https://www.alphavantage.co/query?'
        out += f'function={action}'
        out += f'&symbol={symbol}'
        out += f'&outputsize=full'
        out += f'&slice=year{y}month{m}'
        out += f'&interval=1min'
        #out += "datatype=json"
        #out += f'&adjusted=true',
        #out += "&slice=year1month1",
        out += f'&apikey={av_apikey}'
        
        return out
    
    if input_action == '60min':
        action = "TIME_SERIES_INTRADAY_EXTENDED"
        out = ""
        out += f'https://www.alphavantage.co/query?'
        out += f'function={action}'
        out += f'&symbol={symbol}'
        out += f'&outputsize=full'
        out += f'&slice=year{y}month{m}'
        out += f'&interval=60min'
        #out += "datatype=json"
        #out += f'&adjusted=true',
        #out += "&slice=year1month1",
        out += f'&apikey={av_apikey}'

    return out
    print(f"Invalid Action: {input_action}")
    print(f"No URL Returned, PLease try again")
    return None
    


action = 'daily'
action = '1min'
action = '60min'


for symbol in symbols:
    if (action == '1min') or (action == '60min'):
           
        #//*** initialize output dataframe
        out_df = pd.DataFrame()
            
        #//*** Loop the year
        for year in [1,2]:

            #//*** Loop each month
            for month in range(1,13):
                print(f"Length out_df: {len(out_df)}")
                print(f"Building URL: {symbol} - Month {month} Year {year}")
                url = build_url(action,symbol,month,year)

                print("Downloading")
                print(url)
                r = requests.get(url)
                print(r.text[:1000])
                f = open("t.csv", "w")

                f.write(r.text)
                f.close()

                out_df = pd.concat([out_df,pd.read_csv("t.csv")])

                print("Waiting 20 Seconds")
                time.sleep(20)

                    
        print("df Built")
        print(out_df.head(10))

        output_filename = f"./stocks/{symbol}_{action}.csv.zip"

        #//*** Convert Path to Mac formatting if needed
        #if platform.system() == 'Darwin':
            #output_filename = output_filename.replace("\\","/")

        print(f"Writing dataframe to File: {output_filename}")
        out_df.to_csv(output_filename,compression="zip",index=False)

                        
                    #print(year,month)
            #print("Waiting 20 Seconds")
            #time.sleep(20)

            #//*** Build the Url Request for each symbol
    #//*** Verify we built a proper url
#    if url != None:
        
#        print("Downloading....")
        #//*** Download the data for each Symbol
#        r = requests.get(url)

"""
        #//*** Convert raw string to dictionary for processing 
        data = r.json()

        #//*** Output Dictionary
        out_dict = {}
        print("Processing....")
        #//*** Process Data into the out_dict
        for date in data[data_key]:
            #//*** Build out_dict (output_dictionary) keys 
            if len(out_dict.keys()) == 0:
                out_dict['date'] = []
                out_dict['symbol'] = []

                #//*** Get this dictionary for the first row. Use the key values, but strip the first 3 characters which are numeric
                for key in data[data_key][date].keys():
                    out_dict[key[3:]] = []

            #//*** Add Date to out_dict
            out_dict['date'].append(date)

            #//*** Add Symbol to out_dict
            out_dict['symbol'].append(symbol)

            #//*** Loop through the daily values and append to the out_dict
            for key,value in data[data_key][date].items():

                #//*** Trim first 3 characters off key and append to the appropriate dictionary list
                out_dict[key[3:]].append(value)

        print("Building Dataframe")
        out_df = pd.DataFrame()
        #//*** Convert the Dictionary to a Dataframe
        #//*** Each Key is a column, the data is the list
        for key,value in out_dict.items():
            out_df[key] = value

        #//*** Generic Filename - Placeholder
        output_filename = f".\\stocks\\{symbol}_need_a_better_name.csv.zip"
        
        
        #//*** Build filename based on action type
        if action == 'daily':
            output_filename = f".\\stocks\\{symbol}_daily.csv.zip"
        
        if action == '1min':
            output_filename = f".\\stocks\\{symbol}_1min.csv.zip"

        #//*** Convert Path to Mac formatting if needed
        if platform.system() == 'Darwin':
            output_filename = output_filename.replace("\\","/")
        
        print(f"Writing dataframe to File: {output_filename}")
        out_df.to_csv(output_filename,compression="zip",index=False)    

    #else:
    #    print("We've got an url problem Skipping")
    
    #//*** Wait 20 seconds so we don't hammer the API
    #//*** Max is 5 calls / minute & 500 /day
    
    #print("Waiting 20 Seconds")
    #time.sleep(20)
"""
print("done")




    

Length out_df: 0
Building URL: amc - Month 1 Year 1
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year1month1&interval=60min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2021-07-15 20:00:00,38.3,38.4,37.56,38.1,721940
2021-07-15 19:00:00,38.75,38.85,38.0,38.29,923939
2021-07-15 18:00:00,39.17,39.87,38.25,38.75,3911023
2021-07-15 17:00:00,35.96,39.2,35.76,39.17,4218169
2021-07-15 16:00:00,35.33,36.1,34.67,35.9911,19774603
2021-07-15 15:00:00,33.9389,35.43,33.33,35.3291,16629161
2021-07-15 14:00:00,35.3378,35.35,33.55,33.935,15622930
2021-07-15 13:00:00,35.4289,36.2081,35.1841,35.3371,15245005
2021-07-15 12:00:00,35.73,37.3899,35.2,35.43,34788196
2021-07-15 11:00:00,36.38,37.4,34.25,35.74,46937255
2021-07-15 10:00:00,31.66,36.5,31.43,36.4199,38351647
2021-07-15 09:00:00,32.0,32.3399,31.1,31.66,1481389
2021-07-15 08:00:00,32.0,32.34,31.24,31.6,867082
2021-07-15 07:00:00,32.43,32.43,31.53,32.03,251408
2021-0

Length out_df: 2286
Building URL: amc - Month 8 Year 1
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year1month8&interval=60min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2020-12-17 20:00:00,2.86,2.89,2.85,2.88,47577
2020-12-17 19:00:00,2.86,2.8801,2.85,2.87,42813
2020-12-17 18:00:00,2.87,2.87,2.85,2.855,101794
2020-12-17 17:00:00,2.85,2.87,2.83,2.87,107975
2020-12-17 16:00:00,2.815,2.85,2.8,2.84,3314993
2020-12-17 15:00:00,2.795,2.82,2.78,2.815,1477858
2020-12-17 14:00:00,2.815,2.86,2.76,2.795,2667521
2020-12-17 13:00:00,2.9112,2.92,2.8,2.815,3140582
2020-12-17 12:00:00,2.8934,2.95,2.88,2.915,4410327
2020-12-17 11:00:00,2.775,2.92,2.77,2.8975,4906582
2020-12-17 10:00:00,2.8,2.82,2.74,2.775,3135542
2020-12-17 09:00:00,2.76,2.82,2.75,2.81,288502
2020-12-17 08:00:00,2.81,2.81,2.76,2.76,103187
2020-12-17 07:00:00,2.81,2.81,2.78,2.8,16120
2020-12-17 06:00:00,2.78,2.82,2.77,2.77,1347
2020-12-17 05:00:00,2.8

Length out_df: 4444
Building URL: amc - Month 3 Year 2
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year2month3&interval=60min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2020-05-21 20:00:00,4.65,4.68,4.65,4.66,6423
2020-05-21 19:00:00,4.6,4.65,4.6,4.65,12023
2020-05-21 18:00:00,4.6102,4.62,4.59,4.6199,3875
2020-05-21 17:00:00,4.61,4.63,4.56,4.6,33399
2020-05-21 16:00:00,4.625,4.785,4.57,4.61,1785038
2020-05-21 15:00:00,4.58,4.645,4.58,4.625,449314
2020-05-21 14:00:00,4.5921,4.62,4.57,4.58,341019
2020-05-21 13:00:00,4.61,4.63,4.58,4.6,302358
2020-05-21 12:00:00,4.635,4.635,4.59,4.6,275992
2020-05-21 11:00:00,4.65,4.678,4.58,4.635,526842
2020-05-21 10:00:00,4.67,4.7,4.55,4.6415,799596
2020-05-21 09:00:00,4.6401,4.7,4.6,4.7,17868
2020-05-21 08:00:00,4.64,4.65,4.6,4.61,2923
2020-05-21 07:00:00,4.6,4.6,4.58,4.58,608
2020-05-20 20:00:00,4.66,4.74,4.66,4.74,3166
2020-05-20 19:00:00,4.66,4.67,4.66,4.67,480
20

Length out_df: 6250
Building URL: amc - Month 10 Year 2
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year2month10&interval=60min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2019-10-24 17:00:00,9.30923250848,9.30923250848,9.15407863334,9.15407863334,11186
2019-10-24 16:00:00,9.17347286773,9.34802097726,9.16377575053,9.29953539128,438171
2019-10-24 15:00:00,9.12498728175,9.28983827409,9.11529016455,9.17521834883,475898
2019-10-24 14:00:00,9.11529016455,9.14438151614,9.04741034418,9.12498728175,411581
2019-10-24 13:00:00,9.32377818427,9.32765703115,9.09589593016,9.11529016455,397969
2019-10-24 12:00:00,9.27044403969,9.36741521166,9.22195845371,9.31892962568,206548
2019-10-24 11:00:00,9.34802097726,9.38680944605,9.2413526881,9.26928038563,227580
2019-10-24 10:00:00,9.61954025876,9.62923737596,9.2510498053,9.34802097726,266986
2019-10-23 19:00:00,9.64863161035,9.64863161035,9.64863161035,9.64863161035,150
2

In [67]:
output_filename = f"./stocks/{symbol}_{action}.csv.zip"

#//*** Convert Path to Mac formatting if needed
#if platform.system() == 'Darwin':
    #output_filename = output_filename.replace("\\","/")

print(f"Writing dataframe to File: {output_filename}")
out_df.sort_values('time').to_csv(output_filename,compression="zip",index=False)

Writing dataframe to File: ./stocks/gme_1min.csv.zip


In [87]:
tdf = pd.read_csv(output_filename)
print(len(tdf['time'].unique()))
print(len(tdf))

245627
245627


In [49]:
f = open("t.csv", "w")
f.write(r.text)
f.close()

pd.read_csv("t.csv")


#decoded_content = r.decode('utf-8')
#cr = csv.reader(r.text.splitlines(), delimiter=',')
#cr

,time,open,high,low,close,volume
0,2021-07-12 20:00:00,189.50,189.50,189.50,189.50,417
1,2021-07-12 19:53:00,189.44,189.44,189.44,189.44,118
2,2021-07-12 19:22:00,189.01,189.01,189.01,189.01,102
3,2021-07-12 18:52:00,189.00,189.00,189.00,189.00,250
4,2021-07-12 17:53:00,189.50,189.50,189.50,189.50,163
...,...,...,...,...,...,...
10041,2021-06-14 04:38:00,236.39,236.40,236.39,236.40,975
10042,2021-06-14 04:36:00,236.03,236.03,236.03,236.03,358
10043,2021-06-14 04:21:00,237.50,237.50,237.50,237.50,489
10044,2021-06-14 04:15:00,235.95,235.95,235.95,235.95,418


In [71]:
"""
#//**** INTRA day EXTENDED provides stock data at intervals of 1min, 5min, 15min, 30min, 60min,
#//**** Each query provides one month at a time

import csv
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={symbol}&interval=15min&slice=year1month1&apikey={av_apikey}'

with requests.Session() as s:
    download = s.get(url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)
"""

"\n#//**** INTRA day EXTENDED provides stock data at intervals of 1min, 5min, 15min, 30min, 60min,\n#//**** Each query provides one month at a time\n\nimport csv\nCSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={symbol}&interval=15min&slice=year1month1&apikey={av_apikey}'\n\nwith requests.Session() as s:\n    download = s.get(url)\n    decoded_content = download.content.decode('utf-8')\n    cr = csv.reader(decoded_content.splitlines(), delimiter=',')\n    my_list = list(cr)\n    for row in my_list:\n        print(row)\n"